In [6]:
import os
import random

import numpy as np
import pandas as pd
import scipy.sparse as sps
import matplotlib.pyplot as plt
import seaborn as snb

# Random seed for reproducibility
SEED = 42
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [7]:
from data_manager import DatasetLoader, DatasetSplitter, URMGenerator

dataset_loader = DatasetLoader()
dataset_splitter = DatasetSplitter(dataset_loader)
dataset_train, dataset_val = dataset_splitter.load_train_val()
URM_generator = URMGenerator(dataset_train, dataset_val)
URM_train, URM_val = URM_generator.generate_explicit_URM()
URM_all = URM_train + URM_val

Loading previusly generated splits...
Generating explicit URM...


ValueError: object dtype is not supported by sparse matrices

In [ ]:
from evaluation.evaluator import EvaluatorHoldout

evaluator = EvaluatorHoldout(URM_val, cutoff_list=[10])

In [4]:
from Recommenders.MatrixFactorization.IALSRecommender import (
    IALSRecommender,
)
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

output_folder_path = "result_experiments/IALSRecommender/"
recommender_class = IALSRecommender
n_cases = 10
n_random_starts = int(n_cases * 0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Define hyperparameters
hyperparameters_range_dictionary = {
    "num_factors": Integer(1, 200),
    "epochs": Categorical([300]),
    "confidence_scaling": Categorical(["linear", "log"]),
    "alpha": Real(low=1e-3, high=50.0, prior="log-uniform"),
    "epsilon": Real(low=1e-3, high=10.0, prior="log-uniform"),
    "reg": Real(low=1e-5, high=1e-2, prior="log-uniform"),
}

hyperparameter_search = SearchBayesianSkopt(
    recommender_class,
    evaluator_validation=evaluator,
)

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[
        URM_train,
    ],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)


In [ ]:
hyperparameter_search.search(
    recommender_input_args,
    hyperparameter_search_space=hyperparameters_range_dictionary,
    n_cases=n_cases,
    n_random_starts=n_random_starts,
    save_model="best",
    output_folder_path=output_folder_path,  # Where to save the results
    output_file_name_root=recommender_class.RECOMMENDER_NAME,  # How to call the files
    metric_to_optimize=metric_to_optimize,
    cutoff_to_optimize=cutoff_to_optimize,
)

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'num_factors': 116, 'epochs': 300, 'confidence_scaling': 'log', 'alpha': 0.007639672839788821, 'epsilon': 0.02570397424946322, 'reg': 0.004316111244636997}


KeyboardInterrupt: 

In [6]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path=output_folder_path)
search_metadata = data_loader.load_data(
    recommender_class.RECOMMENDER_NAME + "_metadata.zip"
)

result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [7]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,topK,epochs,symmetric,sgd_mode,lambda_i,lambda_j,learning_rate
0,166,1499,True,adam,0.000124,0.001693,0.000103
1,1324,1499,False,adagrad,0.000032,0.006309,0.016491
2,1191,1499,False,adam,0.002197,0.000027,0.00117
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

In [9]:
exception_list = search_metadata["exception_list"]
exception_list

['Traceback (most recent call last):\n  File "d:\\Developing\\RecSys_PoliMi_challenge_2020\\HyperparameterTuning\\SearchAbstractClass.py", line 604, in _objective_function\n    result_df, recommender_instance = self._evaluate_on_validation(\n  File "d:\\Developing\\RecSys_PoliMi_challenge_2020\\HyperparameterTuning\\SearchAbstractClass.py", line 395, in _evaluate_on_validation\n    self.model_counter: recommender_instance.get_validation_summary_table()\n  File "d:\\Developing\\RecSys_PoliMi_challenge_2020\\Recommenders\\Incremental_Training_Early_Stopping.py", line 110, in get_validation_summary_table\n    return self._earlystopping_validation_summary_df.copy()\nAttributeError: \'NoneType\' object has no attribute \'copy\'\n',
 'Traceback (most recent call last):\n  File "d:\\Developing\\RecSys_PoliMi_challenge_2020\\HyperparameterTuning\\SearchAbstractClass.py", line 604, in _objective_function\n    result_df, recommender_instance = self._evaluate_on_validation(\n  File "d:\\Developin

In [10]:
recommender = recommender_class(URM_all)
recommender.fit(**best_hyperparameters)
recommender.save_model(
    folder_path=output_folder_path,
    file_name=recommender_class.RECOMMENDER_NAME
    + "_best_model_trained_on_everything.zip",
)

TypeError: fit() argument after ** must be a mapping, not NoneType

In [ ]:
from utils.create_submission import create_submission

create_submission(recommender)